In [12]:
from action_language_interpreter.util import solve

In [13]:
time_starts_at_0_axiom = """
time(0).
"""

condition_axioms = """
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, F), not pos_at(F, T).
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, -F), not neg_at(F, T).
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, unk(F)), not unk_at(F, T).
"""

causes_axioms = """
% E causes F if C*
pos_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, F, C), occ_at(E, T), not conditionFails_at(C, T).
neg_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, -F, C), occ_at(E, T), not conditionFails_at(C, T).
unk_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, unk(F), C), occ_at(E, T), not conditionFails_at(C, T).
"""

impossible_if_axiom = """
% E impossible_if C*
:- event(E), time(T), eventImpossible_if(E, C), occ_at(E, T), not conditionFails_at(C, T).
"""

if_axioms = """
% F if C*
pos_at(F, T) :- fluent(F), time(T), fluent_if(F, C), not conditionFails_at(C, T).
neg_at(F, T) :- fluent(F), time(T), fluent_if(-F, C), not conditionFails_at(C, T).
unk_at(F, T) :- fluent(F), time(T), fluent_if(unk(F), C), not conditionFails_at(C, T).
"""

inertia_axioms = """
pos_at(F, T+1) :- fluent(F), time(T), time(T+1), pos_at(F, T), not neg_at(F, T+1), not unk_at(F, T+1).
neg_at(F, T+1) :- fluent(F), time(T), time(T+1), neg_at(F, T), not pos_at(F, T+1), not unk_at(F, T+1).
unk_at(F, T+1) :- fluent(F), time(T), time(T+1), unk_at(F, T), not pos_at(F, T+1), not neg_at(F, T+1).
"""

consistency_axioms = """
:- fluent(F), time(T), pos_at(F, T), unk_at(F, T).
:- fluent(F), time(T), neg_at(F, T), unk_at(F, T).
"""

completion_g1_axiom = """
% g1
pos_at(F, 0) :- fluent(F), init(F).
neg_at(F, 0) :- fluent(F), init(-F).
"""

completion_g2_axiom = """
% g2
pos_at(F, 0) :- fluent(F), forced(F), default(F), not init(-F).
1 { pos_at(F, 0); neg_at(F, 0) } 1 :- fluent(F), forced(F), not default(F), not init(F), not init(-F).
"""

completion_g3_axiom = """
% g3
neg_at(F, 0) :- fluent(F), default(F), not pos_at(F, 0).
unk_at(F, 0) :- fluent(F), not default(F), not pos_at(F, 0), not neg_at(F, 0).
"""

axioms = (time_starts_at_0_axiom, condition_axioms, causes_axioms, impossible_if_axiom, if_axioms, inertia_axioms,
          consistency_axioms, completion_g1_axiom, completion_g2_axiom, completion_g3_axiom)

In [14]:
force_all_fluents = """

forced(F) :- fluent(F).

"""

In [15]:
force_all_non_default = """

forced(F) :- fluent(F), not default(F).

"""

In [16]:
reason_by_case_for_all_fluents = """

split_at(F, T) :- fluent(F), occ_at(_, T).

"""

In [17]:
instance = """

%% Steps
time(0..1).

%% Events
event(d). % John goes on a first date with Mary.

%% Fluents
fluent(m). % John is married.
fluent(ab). % John is abnormal.

%% Knowledge Base
% Action Description
eventImpossible_if(d, 1).
condition_fluent(1, m).
condition_fluent(1, -ab).

% Default fluents
default(ab).

%% Query
query(m).

"""

In [18]:
action_sequence = """
occ_at(d, 0).
"""

In [20]:
solve([*axioms,
       instance,
       action_sequence,
       force_all_non_default,
       reason_by_case_for_all_fluents], clingo_args=("--models", "0", "--enum-mode", "cautious"));

Answer  1: { default(ab) event(d) fluent(ab) fluent(m) forced(m) query(m) time(0) time(1) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) neg_at(m,0) neg_at(m,1) occ_at(d,0) split_at(ab,0) split_at(m,0) }.
SAT 1 


<block>:3:28-35: info: atom does not occur in any rule head:
  init(F)

<block>:3:37-52: info: atom does not occur in any rule head:
  fluent_if(F,C)

<block>:3:49-73: info: atom does not occur in any rule head:
  event_causes_if(E,F,C)

<block>:3:55-63: info: atom does not occur in any rule head:
  init((-F))

<block>:4:28-36: info: atom does not occur in any rule head:
  init((-F))

<block>:4:37-53: info: atom does not occur in any rule head:
  fluent_if((-F),C)

<block>:4:49-74: info: atom does not occur in any rule head:
  event_causes_if(E,(-F),C)

<block>:4:81-88: info: atom does not occur in any rule head:
  init(F)

<block>:4:94-102: info: atom does not occur in any rule head:
  init((-F))

<block>:5:37-57: info: atom does not occur in any rule head:
  fluent_if(unk(F),C)

<block>:5:49-78: info: atom does not occur in any rule head:
  event_causes_if(E,unk(F),C)



In [22]:
# Answer set from above as facts.
R = """

default(ab).
event(d).
fluent(ab).
fluent(m).
forced(m).
query(m).
time(0).
time(1).
conditionFails_at(1,0).
conditionFails_at(1,1).
condition_fluent(1,m).
condition_fluent(1,-ab).
eventImpossible_if(d,1).
neg_at(ab,0).
neg_at(ab,1).
neg_at(m,0).
neg_at(m,1).
occ_at(d,0).
split_at(ab,0).
split_at(m,0).

"""

In [25]:
make_conservative_assumptions = """

init(F) :- fluent(F), forced(F), pos_at(F, 0).
init(-F) :- fluent(F), forced(F), neg_at(F, 0).

#show init/1.

"""

In [26]:
solve([R, make_conservative_assumptions]);

Answer  1: { init(-m) }.
SAT 1 


<block>:3:34-46: info: atom does not occur in any rule head:
  pos_at(F,0)



In [ ]:
initial_knowledge = """

init(-m).

"""